In [15]:
import os 
import pathlib

import sumo_rl
from sumo_rl.environment.env import env, parallel_env, SumoEnvironment

import ray

from ray.tune import register_env
from ray.rllib.env import PettingZooEnv, ParallelPettingZooEnv
from ray.rllib.algorithms import ppo 
from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility

from environment.envs import RealMultiAgentSumoEnv
from environment.observation import Grid2x2ObservationFunction, EntireObservationFunction
from environment.reward_functions import combined_reward

from pettingzoo.utils import wrappers

os.environ["PYTHONWARNINGS"] = 'ignore::DeprecationWarning'

In [2]:
os.environ['SUMO_HOME'] = '/opt/homebrew/opt/sumo/share/sumo'
os.environ['RAY_PICKLE_VERBOSE_DEBUG'] = '1'

env_folder = "data/2x2grid"

env_name = "2x2grid"

multi_agent_env = parallel_env(    
        net_file = os.path.join(env_folder, "2x2.net.xml"),
        route_file = os.path.join(env_folder, "2x2.rou.xml"),
        reward_fn = combined_reward,
        observation_class = EntireObservationFunction, 
        out_csv_name="outputs/2x2grid/ppo", 
        num_seconds=1000000,
        add_per_agent_info=True,
        add_system_info=True,
        single_agent=False)

seed = 4

env_params = {"net_file": os.path.join(env_folder, "2x2.net.xml"),
        "route_file": os.path.join(env_folder, "2x2.rou.xml"),
        "reward_fn": combined_reward,
        "observation_class": EntireObservationFunction, 
        "out_csv_name": "outputs/2x2grid/ppo", 
        "num_seconds": 1000,
        "add_per_agent_info": True,
        "add_system_info": True,
        "sumo_seed": seed,
        "single_agent": False}

ray.shutdown()
ray.init()

# ValueError: Env must be of one of the following supported types: 
# BaseEnv, gymnasium.Env, gym.Env, MultiAgentEnv, VectorEnv, RemoteBaseEnv, ExternalMultiAgentEnv, 
# ExternalEnv, but instead is of type <class 'environment.envs.RealMultiAgentSumoEnv'>.

multi_agent_par_env = RealMultiAgentSumoEnv(**env_params) # SUMO environment implementing PettingZoo API TODO: CHANGE NAME

rllib_compat_ppz_env_aecrllib_compat_ppz_env = ParallelPettingZooEnv(multi_agent_par_env) # Wrap it to be a Parallel Petting Zoo env 

rllib_compat_env = MultiAgentEnvCompatibility(rllib_compat_ppz_env) # convert from old api to new 

register_env(name=env_name, env_creator= lambda config : rllib_compat_env) # register env

 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 16ms, vehicles TOT 0 ACT 0 BUF 0)                     


/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages/pettingzoo/utils/conversions.py:132: UserWarning: The base environment `sumo_rl_v0` does not have a `render_mode` defined.
  warnings.warn(
2024-02-18 08:27:15,598	INFO worker.py:1673 -- Started a local Ray instance.


 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 14ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds


In [3]:
ray.rllib.utils.check_env(rllib_compat_env)

ValueError: Traceback (most recent call last):
  File "/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages/ray/rllib/utils/pre_checks/env.py", line 307, in check_multiagent_environments
    obs_and_infos = env.reset(seed=42, options={})
  File "/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages/ray/rllib/env/wrappers/multi_agent_env_compatibility.py", line 46, in reset
    self.env.seed(seed)
AttributeError: 'ParallelPettingZooEnv' object has no attribute 'seed'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages/ray/rllib/utils/pre_checks/env.py", line 81, in check_env
    check_multiagent_environments(env)
  File "/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages/ray/rllib/utils/pre_checks/env.py", line 312, in check_multiagent_environments
    raise ValueError(
ValueError: Your environment (<MultiAgentEnvCompatibilit

In [ ]:
config = (ppo.PPOConfig()
        .environment(env_name))

algo = config.build()

 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 11ms, vehicles TOT 0 ACT 0 BUF 0)                     


/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This AP

(RolloutWorker pid=93203)  Retrying in 1 seconds
(RolloutWorker pid=93202) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 14ms, vehicles TOT 0 ACT 0 BUF 0)                     


(RolloutWorker pid=93202) 2024-02-14 10:44:29,338	WARNING __init__.py:10 -- PG has/have been moved to `rllib_contrib` and will no longer be maintained by the RLlib team. You can still use it/them normally inside RLlib util Ray 2.8, but from Ray 2.9 on, all `rllib_contrib` algorithms will no longer be part of the core repo, and will therefore have to be installed separately with pinned dependencies for e.g. ray[rllib] and other packages! See https://github.com/ray-project/ray/tree/master/rllib_contrib#rllib-contrib for more information on the RLlib contrib effort.


(RolloutWorker pid=92364) Error: tcpip::Socket::recvAndCheck @ recv: peer shutdown
(RolloutWorker pid=92364) Quitting (on error).


(RolloutWorker pid=92364) Step #0.00


In [11]:
multi_agent_par_env = RealMultiAgentSumoEnv(**env_params) # SUMO environment implementing PettingZoo API TODO: CHANGE NAME

rllib_compat_ppz_env_aec = PettingZooEnv(multi_agent_par_env) # Wrap it to be a Petting Zoo env 

ray.rllib.utils.check_env(rllib_compat_ppz_env_aec)

 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 51ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds


In [18]:
from pettingzoo.utils.conversions import aec_to_parallel_wrapper

In [19]:
multi_agent_aec_pz_env = RealMultiAgentSumoEnv(**env_params) # my own subclass inheriting from SumoEnvironmentPZ (a class that implements PettingZoo API)

multi_agent_aec_pz_env_asserted = wrappers.AssertOutOfBoundsWrapper(multi_agent_aec_pz_env)
multi_agent_aec_pz_env_order_enfor = wrappers.OrderEnforcingWrapper(multi_agent_aec_pz_env_asserted)

multi_agent_par_env_order_enfor_par = aec_to_parallel_wrapper(multi_agent_aec_pz_env_order_enfor)

rllib_compat_ppz_env_par = ParallelPettingZooEnv(multi_agent_par_env_order_enfor_par) # Wrap it to be a Parallel Petting Zoo env 

ray.rllib.utils.check_env(rllib_compat_ppz_env_par)

 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 17ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds


/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages/pettingzoo/utils/conversions.py:132: UserWarning: The base environment `sumo_rl_v0` does not have a `render_mode` defined.
  warnings.warn(
/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages/pettingzoo/utils/conversions.py:144: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn(
/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages/pettingzoo/utils/conversions.py:158: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn(


Step #0.00 (0ms ?*RT. ?UPS, TraCI: 42ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds


In [14]:
# rllib_compat_env = MultiAgentEnvCompatibility(rllib_compat_ppz_env) # convert from old api to new 

# env = wrappers.AssertOutOfBoundsWrapper(env)
# env = wrappers.OrderEnforcingWrapper(env)

In [11]:
a = (lambda agent_id, *args, **kwargs: agent_id)
b = (lambda agent_id, **kwargs: agent_id)

In [29]:
a(3, 8, k=8, j=7)

3

In [31]:
lamda_func = (lambda integ : 4)

In [40]:
lamda_func(6)

4